# 1. Бизнес-анализ
## 1.1 Цель задачи
Предсказание качества вина на основании его физико-химических характеристик

### 1.2 Доступные данные
На основании описания датасета на сайте Kaggle (см. ссылку выше), а также других открытых источников становится понятно, что текущий набор данных содержит информацию только о физико-химических характеристиках Португальских белых и красных вин Vinho Verde, полученных путем проведения различных лабораторных измерений и тестов. Также для каждого вина представлена информация о его типе (белое или красное) и о том, какую оценку по десятибалльной шкале вино получило от сомелье.
Каждый из измеренных в лаборатории параметров так или иначе влияет на вкусовые, цветовые и другие характеристики вин.
Степень влияния каждого из признаков может быть оценена в последующем анализе данных.
В описании к датасету также особо отмечено, что данные в выборке не являются нормализованными. Значительная часть вин, описанных в датасете, относится к винам среднего качества, а вина плохого или высшего качества встречаются в выборке редко. Это уточнение важно учесть при анализе выбросов в данных, которые с большой долей вероятности могут оказаться не выбросами, а значимыми данными.
Также в описании данных указано, что все представленные данные являются релевантными, и поэтому интересными для анализа могут оказаться методы отбора признаков.





# 2. Анализ данных

## 2.1 Импорт библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn. metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier

## 2.2 Загрузка данных

In [21]:
# загружаем данные в датафрейм
df = pd.read_csv('winequalityN.csv', sep = ',')

In [22]:
df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


## 2.3 Описание переменных

1 - fixed acidity

2 - volatile acidity

3 - citric acid

4 - residual sugar

5 - chlorides

6 - free sulfur dioxide

7 - total sulfur dioxide

8 - density

9 - pH

10 - sulphates

11 - alcohol

Output variable (based on sensory data):

12 - quality (score between 0 and 10)

# 3 Подготовка данных

In [4]:
# проверим данные на пропуски
df.isna().sum()

type                     0
fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [5]:
# пропуски в данных в процентах
(df.isna().mean() * 100).round(2)

type                    0.00
fixed acidity           0.15
volatile acidity        0.12
citric acid             0.05
residual sugar          0.03
chlorides               0.03
free sulfur dioxide     0.00
total sulfur dioxide    0.00
density                 0.00
pH                      0.14
sulphates               0.06
alcohol                 0.00
quality                 0.00
dtype: float64

In [6]:
# удаляем пропуски
df = df[pd.isnull(df['fixed acidity']) == 0]
df = df[pd.isnull(df['volatile acidity']) == 0]
df = df[pd.isnull(df['citric acid']) == 0]
df = df[pd.isnull(df['residual sugar']) == 0]
df = df[pd.isnull(df['chlorides']) == 0]
df = df[pd.isnull(df['pH']) == 0]
df = df[pd.isnull(df['sulphates']) == 0]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6463 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6463 non-null   object 
 1   fixed acidity         6463 non-null   float64
 2   volatile acidity      6463 non-null   float64
 3   citric acid           6463 non-null   float64
 4   residual sugar        6463 non-null   float64
 5   chlorides             6463 non-null   float64
 6   free sulfur dioxide   6463 non-null   float64
 7   total sulfur dioxide  6463 non-null   float64
 8   density               6463 non-null   float64
 9   pH                    6463 non-null   float64
 10  sulphates             6463 non-null   float64
 11  alcohol               6463 non-null   float64
 12  quality               6463 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 706.9+ KB


In [8]:
# переведем категориальную переменную type в значения 0 и 1
df = pd.get_dummies(df, columns = ['type'])

In [9]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type_red,type_white
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,0,1
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,0,1
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,0,1
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0,1
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1,0
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1,0
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1,0
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1,0


In [10]:
# посмотрим на баланс классов
df.groupby('quality').size()

quality
3      30
4     214
5    2128
6    2820
7    1074
8     192
9       5
dtype: int64

In [11]:
list(df)

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality',
 'type_red',
 'type_white']

In [12]:
# вектор признаков
X = df[['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'type_red',
 'type_white']]

In [13]:
# целевая переменная
y = df['quality']

In [14]:
# разбиваем данные на обучающую и тестовую выборки в соотношении 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Моделирование

In [15]:
# обучаем модель логистической регрессии
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [16]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.55      0.12      0.20        48
           5       0.61      0.57      0.59       430
           6       0.50      0.71      0.59       540
           7       0.51      0.27      0.36       228
           8       0.00      0.00      0.00        42
           9       0.00      0.00      0.00         1

    accuracy                           0.54      1293
   macro avg       0.31      0.24      0.25      1293
weighted avg       0.52      0.54      0.51      1293



C:\Anac\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anac\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anac\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# выровняем баланс классов
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [18]:
# снова разбиваем данные на обучающую и тестовую выборки в соотношении 80/20
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [19]:
# обучаем модель
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           3       0.47      0.48      0.48       586
           4       0.47      0.47      0.47       591
           5       0.42      0.45      0.44       551
           6       0.36      0.25      0.29       568
           7       0.37      0.29      0.32       536
           8       0.43      0.53      0.47       564
           9       0.82      1.00      0.90       552

    accuracy                           0.49      3948
   macro avg       0.48      0.50      0.48      3948
weighted avg       0.48      0.49      0.48      3948



In [20]:
# поробуем другой классификатор. Используем случайный лес
model_rf = RandomForestClassifier(random_state = 42)
model_rf.fit(X_train, y_train)
predictions = model_rf.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           3       1.00      1.00      1.00       586
           4       0.99      1.00      0.99       591
           5       0.82      0.84      0.83       551
           6       0.81      0.69      0.75       568
           7       0.87      0.96      0.91       536
           8       0.99      1.00      1.00       564
           9       1.00      1.00      1.00       552

    accuracy                           0.93      3948
   macro avg       0.93      0.93      0.93      3948
weighted avg       0.93      0.93      0.93      3948



# 5. Оценка модели

Удалось добиться accuracy = 0.93 с использованием RandomOverSampler для выравнивания баланса классов и RandomForestClassifier в качестве классификатора

# 6. Развертывание
